# Splink v4 Deduplication Demo

This demo shows the steps to deduplicate a sample data set along with examples of visualisations splink provides.

The demo code is based on the exampe [Deduplicate 50k rows historical persons](https://moj-analytical-services.github.io/splink/demos/examples/duckdb/deduplicate_50k_synthetic.html) in the official splink documentation with changes by Barney Lawrence to expand on the concepts demonstrated and to adapt the code to run in a Microsoft Fabric or Synapse Analytics environment.

## Install Splink
For convenience splink is installed direct via splink.

In a production environment splink should be pre-loaded at a defined version number to avoid issues with unexpected version changes. Similarly inside a private network it may not be possible to access the online versions of the library and its dependencies.

This is easily done in Fabric with a custom environment and less easily done in Synapse Analytics by extracting the relevant WHL files and manually adding them to the workspace and spark pool.

In [ ]:
%pip install splink


## Setup
Load in libraries and some initialisation to create the file paths splink needs to work in a Microsoft environment.

Suppressing deprecation warnings as splink sometimes uses older functions and can trigger a lot of warnings in the outputs that we can't do anything about.

In [ ]:
from splink import DuckDBAPI, SparkAPI

db_api = DuckDBAPI()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

spark.sparkContext.setCheckpointDir("Files/tmp_checkpoints")

import os

os.makedirs("Files/TempReports", exist_ok=True)

## Sample Data

Reading in the historical 50k sample data set included with splink.

In [ ]:
from splink import splink_datasets

df = splink_datasets.historical_50k

df[:1000]

## Profile Columns
This visualisation gives an example of the tools available in splink to support initial exploration of a data set.

In this case we generate profiling of a defined set of columns to understand high and low cardinality values in them.

In [ ]:
from splink.exploratory import profile_columns

profile_columns(df, db_api,
    column_expressions=["first_name", "surname", "occupation", "substr(dob, 1,4)"], 
    top_n=10, bottom_n=10
)
    

## Blocking Rules
Blocking rules define the blocks of rows that will be compared rather than running a full cartesian product of the data.

Blocking rules can be defined within the main settings object but it's neater to produce them separately and this also allows for pre analysis of their impact on performance.

The chart generated shows how each rule adds to the cumulative total of distinct pairs compared by the process.

In [ ]:
from splink import block_on
from splink.blocking_analysis import (
    cumulative_comparisons_to_be_scored_from_blocking_rules_chart,
)

blocking_rules = [
    block_on("substr(first_name,1,3)", "substr(surname,1,4)"),
    block_on("surname", "dob"),
    block_on("first_name", "dob"),
    block_on("postcode_fake", "first_name"),
    block_on("postcode_fake", "surname"),
    block_on("dob", "birth_place"),
    block_on("substr(postcode_fake,1,3)", "dob"),
    block_on("substr(postcode_fake,1,3)", "first_name"),
    block_on("substr(postcode_fake,1,3)", "surname"),
    block_on("substr(first_name,1,2)", "substr(surname,1,2)", "substr(dob,1,4)"),
]

cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    table_or_tables=df,
    blocking_rules=blocking_rules,
    db_api=db_api,
    link_type="dedupe_only",
)

## Comparisons

Define the comparisons splink will make between fields within record pairs to assess the probability of a match.

Comparisons can use included pre-defined comparison templates,  be custom built from sets of pre-defined levels or coded directly as SQL syntax.

Each comparison can all be defined directly in the group of comparisons or be defined as a variable beforehand.

In [ ]:
import splink.comparison_library as cl

# Needed to apply term frequencies to first+surname comparison
df["first_name_surname_concat"] = df["first_name"] + " " + df["surname"]

NameComparison = cl.ForenameSurnameComparison(
            "first_name",
            "surname",
            forename_surname_concat_col_name="first_name_surname_concat",
        )

comparisons = [
        NameComparison,
        cl.DateOfBirthComparison(
            "dob", input_is_string=True
        ),
        cl.PostcodeComparison("postcode_fake"),
        cl.ExactMatch("birth_place").configure(term_frequency_adjustments=True),
        cl.ExactMatch("occupation").configure(term_frequency_adjustments=True),
    ]

print(NameComparison.get_comparison("duckdb").human_readable_description)

## Settings

Apply blocking rules, comparisons and other settings to define the needed parameters for the linking process.

These are then applied to create the linker. In this case we only add a single data set as we are de-duplicating within it rather than matching between multiple data sets.

In [ ]:
from splink import Linker, SettingsCreator

settings = SettingsCreator(
    link_type= "dedupe_only",
    blocking_rules_to_generate_predictions=blocking_rules,
    comparisons=comparisons,
    retain_matching_columns= True,
    retain_intermediate_calculation_columns= True,
)

linker = Linker(df, settings, db_api=db_api)

## Estimate Probability Two Random Records Match

Generate a baseline understanding of how many matches are expected within the data by specifying deterministic rules and an approximate proportion of matches they will find.

In [ ]:
deterministic_rules = [
        "l.first_name = r.first_name and l.surname = r.surname and l.dob = r.dob",
        "substr(l.first_name,1,2) = substr(r.first_name,1,2) and l.surname = r.surname and substr(l.postcode_fake,1,2) = substr(r.postcode_fake,1,2)",
        "l.dob = r.dob and l.postcode_fake = r.postcode_fake",
    ]

linker.training.estimate_probability_two_random_records_match(deterministic_rules, recall=0.6)

## Estimate U Using Random Sampling
As randomly picked record pairs are highly unlikely to form a true match we can use these to calculate the majority of U values.

In [ ]:
linker.training.estimate_u_using_random_sampling(max_pairs=5e6)

## Train M from Label Column
If the data contains a label column (any id that uniquely identifies an individual such as National Insurance Number, NHS Number or another shared person id) it can me used to estimate the majority of M values.

For demonstration purposes skip this step to get a better demonstration of expectation maximisation.

In [ ]:
#linker.training.estimate_m_from_label_column("cluster")

## Train Using Expectation Maximisation

M and U values can be trained without label fields by defining blocking rules to generate record pairs and studying the comparison levels for other fields.

Multiple attempts are likely needed to gain a full picture. The first block of code below fails to train for name because part of full name is in every blocking rule. THe second code block fixes this by leaving both first name and surname unconstrained.


In [ ]:
training_blocking_rule = block_on("surname","first_name","dob")
training_session_1 = linker.training.estimate_parameters_using_expectation_maximisation(training_blocking_rule)

training_blocking_rule = block_on("surname","first_name","postcode_fake")
training_session_2 = linker.training.estimate_parameters_using_expectation_maximisation(training_blocking_rule)

training_blocking_rule = block_on("first_name","postcode_fake","dob")
training_session_3 = linker.training.estimate_parameters_using_expectation_maximisation(training_blocking_rule)

training_blocking_rule = block_on("surname","postcode_fake","dob")
training_session_4 = linker.training.estimate_parameters_using_expectation_maximisation(training_blocking_rule)

In [ ]:
training_blocking_rule = block_on("dob","occupation")
training_session_6 = linker.training.estimate_parameters_using_expectation_maximisation(training_blocking_rule)

## Match Weights Chart
This visual shows the final trained result giving weightings for each comparison level of every comparison.

In [ ]:
linker.visualisations.match_weights_chart()

## Term Frequencies Chart
This chart shows details of values where an adjustment to a weighting will be made because the specific field value is either more or less common than average.

In [ ]:
linker.visualisations.tf_adjustment_chart("occupation")

## Predict Results
Apply the calculated weightings to all record pairs generated through blocking rules. Threshold prevents low weighted pairs from being included in the output.

Results are a simple data frame with input columns plus weighting, probability and other columns related to the matching process.

In [ ]:
results = linker.inference.predict(threshold_match_probability=0.5)

In [ ]:
display(results.as_pandas_dataframe(limit=1000))

## Comparison Viewer Dashboard

This dashboard shows all combinations of comparison levels observed in the results ordered by weight. For each an example matched pair can be studied.

Results can be saved as an html file and shared.

In [ ]:

linker.visualisations.comparison_viewer_dashboard(results, "Files/TempReports/comparisons.html", overwrite=True)

f = open('Files/TempReports/comparisons.html', 'r')

displayHTML(f.read())


## Generate Clusters

Based on weightings, clusters of records considered to represent the same entity can be created grouped by a cluster id.

In [ ]:
clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(results, threshold_match_probability=0.95)

In [ ]:
display(clusters.as_pandas_dataframe(limit=1000).sort_values('cluster_id'))

## Cluster Studio Dashboard
This dashboard visualises records within a cluster and the relationships between them.

In [ ]:

linker.visualisations.cluster_studio_dashboard(
    results,
    clusters,
    "Files/TempReports/50k_cluster.html",
    sampling_method="by_cluster_size",
    overwrite=True,
)

f = open('Files/TempReports/50k_cluster.html', 'r')

displayHTML(f.read())